<a href="https://colab.research.google.com/github/nicollesernat/Kaggle-/blob/main/01_exploraci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 01 — Exploración de Datos


In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

In [ ]:
df = pd.read_csv('train.csv')
df.head(20)

## Parámetros

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

#Definir como se va a llamar el archivo dentro del codigo
CSV_PATH = globals().get("CSV_PATH", "train.csv")

#Columnas de interes
ID_COL_INDEX = 0
LOCALIDAD_COL_INDEX = 3
TARGET_COL_INDEX = 16

## Reporte de datos y de figuras

In [ ]:

import re
from pathlib import Path

def running_in_colab():
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False

print("Sugerencias:")
print(" - En Colab, usa rutas como '/content/outputs' o '/content/drive/MyDrive/tu_carpeta'.")
print(" - Rutas de Windows como 'C:\\Users\\...' no existen en Colab (son de tu PC).")

out_dir_input = input("Carpeta de salida (vacío = './outputs'): ").strip()
if not out_dir_input:
    out_dir_input = "outputs"

#Definir la ruta de exportacion

  #Si se define una ruta de Windows, no es accesible desde la ruta de google Colabs

if running_in_colab():
    if re.match(r'^[A-Za-z]:\\\\', out_dir_input):
        print("⚠ Ruta de Windows detectada. No es accesible desde Colab.")
        print("   Se usará '/content/outputs' por defecto. Si quieres Drive, ingresa una ruta que empiece por '/content/drive/MyDrive'.")
        out_dir_input = "/content/outputs"

  #Por otro lado Drive es una buena ruta para guardar las exportaciones del archivo
    if out_dir_input.startswith("/content/drive"):
        try:
            from google.colab import drive  # type: ignore
            drive.mount('/content/drive', force_remount=False)
            print("Google Drive montado en /content/drive")
        except Exception as e:
            print("No fue posible montar Google Drive:", e)

OUT_DIR = Path(out_dir_input).expanduser().resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("Usando carpeta de salida:", OUT_DIR)
SAVE_FIGS = True

def _safe_name(text):
    import re as _re
    text = str(text)
    text = _re.sub(r"[^\\w\\-]+", "_", text).strip("_")
    return text[:60] if len(text) > 60 else text

def save_current_fig(name_hint, dpi=120):
    if SAVE_FIGS:
        try:
            fname = OUT_DIR / f"{_safe_name(name_hint)}.png"
            OUT_DIR.mkdir(parents=True, exist_ok=True)
            plt.savefig(fname, bbox_inches="tight", dpi=dpi)
            print("Figura guardada en:", fname)
        except Exception as e:
            print("⚠️ No se pudo guardar la figura:", e)


## Preparando la exportación

In [ ]:

import os

def listar_out_dir():
    print("Contenido de", OUT_DIR, ":")
    for root, dirs, files in os.walk(OUT_DIR):
        for f in files:
            print(" -", Path(root) / f)

def descargar_zip_outputs():
    if not running_in_colab():
        print("Descarga ZIP disponible solo en Colab.")
        return
    import shutil
    from google.colab import files  # type: ignore
    zip_base = str(OUT_DIR)
    zip_path = shutil.make_archive(zip_base, 'zip', root_dir=OUT_DIR)
    print("ZIP creado en:", zip_path)
    files.download(zip_path)


## Carga de datos

In [ ]:
#Para verificar que el archivo si se abra y se esten visualizando todas las columnas
csv_path = Path(CSV_PATH)
assert csv_path.exists(), f"No se encontró el archivo: {csv_path}"

df = pd.read_csv(csv_path, low_memory=False)
print("Dimensiones (filas, columnas):", df.shape)
display(df.head(11))


## Identificacion de Columnas de interes y de "mayor valor"

In [ ]:

def resolve_column(df, index_fallback, name_override):
    if name_override and name_override in df.columns:
        return name_override
    if 0 <= index_fallback < len(df.columns):
        return df.columns[index_fallback]
    raise ValueError("No se pudo resolver la columna. Ajusta los parámetros.")

ID_COL = resolve_column(df, ID_COL_INDEX, ID_COL_NAME)
LOCALIDAD_COL = resolve_column(df, LOCALIDAD_COL_INDEX, LOCALIDAD_COL_NAME)
TARGET_COL = resolve_column(df, TARGET_COL_INDEX, TARGET_COL_NAME)

print("ID_COL      →", ID_COL)
print("LOCALIDAD   →", LOCALIDAD_COL)
print("TARGET_COL  →", TARGET_COL)


## Información general (Tipo de datos dentro del archivo)


In [ ]:
#Que tipo de variable contiene cada columna, nos dara una idea de si puede hacerse analisis cuantitativo o cualitativo.

display(df.describe(include='all').transpose().head(20))
print("\nTipos de datos:")
print(df.dtypes.head(30))


## Valores nulos por columna

In [ ]:
#Definición de valores nulas por columna, para ver que datos pueden ser despreciados por no contener la información suficiente


null_counts = df.isna().sum().sort_values(ascending=False)
null_pct = (null_counts / len(df) * 100).round(2)
null_summary = pd.DataFrame({"nulos": null_counts, "% nulos": null_pct})
display(null_summary.head(30))

top_n = 20
subset = null_pct.head(top_n)
plt.figure()
cmap = plt.get_cmap('viridis')
colors = cmap(np.linspace(0, 1, len(subset)))
subset.plot(kind="bar", color=colors)
plt.title("% de valores nulos por columna (Top 20)")
plt.xlabel("Columna")
plt.ylabel("% nulos")
plt.tight_layout()
save_current_fig("nulos_top20")
plt.show()


## Distribución de la variable de Clase

In [ ]:

if TARGET_COL in df.columns:
    vc_target = df[TARGET_COL].value_counts(dropna=False)
    display(vc_target.to_frame(name="conteo"))
    plt.figure()
    cmap = plt.get_cmap('viridis')
    colors = cmap(np.linspace(0, 1, len(vc_target)))
    vc_target.plot(kind="bar", color=colors)
    plt.title("Distribución de la clase")
    plt.xlabel("Clase")
    plt.ylabel("Conteo")
    plt.tight_layout()
    save_current_fig("distribucion_target")
    plt.show()
else:
    print("La columna TARGET_COL no está en el DataFrame.")


## Distribución por Localidad

In [ ]:
#Como son tantas localidades decidimos mostrar solo 10. LAs 10 mas representativas.


if LOCALIDAD_COL in df.columns:
    vc_loc = df[LOCALIDAD_COL].value_counts(dropna=False).head(10)
    display(vc_loc.to_frame(name="conteo"))
    plt.figure()
    cmap = plt.get_cmap('viridis')
    colors = cmap(np.linspace(0, 1, len(vc_loc)))
    vc_loc.plot(kind="bar", color=colors)
    plt.title("Estudiantes por Localidad (Top 10)")
    plt.xlabel("Localidad")
    plt.ylabel("Conteo")
    plt.tight_layout()
    save_current_fig("localidad_top10")
    plt.show()
else:
    print("La columna LOCALIDAD_COL no está en el DataFrame.")


##ANALISIS ESTADISTICO

In [ ]:
#Histogramas

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Evitar usar la columna ID si es numérica pero solo identificador
if ID_COL in numeric_cols:
    numeric_cols = [c for c in numeric_cols if c != ID_COL]

max_plots = 6
for col in numeric_cols[:max_plots]:
    plt.figure()
    data = df[col].dropna().values
    n, bins, patches = plt.hist(data, bins=30)
    cmap = plt.get_cmap('plasma')
    for i, p in enumerate(patches):
        p.set_facecolor(cmap(i / max(1, len(patches) - 1)))
    plt.title(f"Histograma — {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    save_current_fig(f"hist_{col}")
    plt.show()

if len(numeric_cols) == 0:
    print("No se detectaron columnas numéricas para histogramas.")
elif len(numeric_cols) > max_plots:
    print(f"Se muestran solo {max_plots} histogramas de {len(numeric_cols)} columnas numéricas detectadas.")


In [ ]:
#Boxplots

max_plots_box = 6
cmap_box = plt.get_cmap('cividis')

for idx, col in enumerate(numeric_cols[:max_plots_box]):
    plt.figure()
    ax = df[[col]].boxplot(patch_artist=True)
    try:
        for i, artist in enumerate(ax.artists):
            artist.set_facecolor(cmap_box(idx / max(1, max_plots_box - 1)))
    except Exception:
        pass
    plt.title(f"Boxplot — {col}")
    plt.xlabel("")
    plt.ylabel(col)
    plt.tight_layout()
    save_current_fig(f"box_{col}")
    plt.show()

# Boxplots por clase
if TARGET_COL in df.columns:
    try:
        n_classes = df[TARGET_COL].nunique(dropna=False)
        if 2 <= n_classes <= 20 and len(numeric_cols) > 0:
            for idx, col in enumerate(numeric_cols[:3]):
                plt.figure()
                ax = df[[col, TARGET_COL]].boxplot(column=col, by=TARGET_COL, grid=False, patch_artist=True)
                try:
                    cmap_class = plt.get_cmap('tab20')
                    for i, artist in enumerate(ax.artists):
                        artist.set_facecolor(cmap_class(i / max(1, len(ax.artists) - 1)))
                except Exception:
                    pass
                plt.suptitle("")
                plt.title(f"Boxplot — {col} por {TARGET_COL}")
                plt.xlabel(TARGET_COL)
                plt.ylabel(col)
                plt.tight_layout()
                save_current_fig(f"box_{col}_por_{TARGET_COL}")
                plt.show()
        else:
            print(f"Se omitieron boxplots por clase (TARGET_COL con {n_classes} clases).")
    except Exception as e:
        print("No fue posible generar boxplots por clase:", e)


##Almacenamiento de información

In [ ]:

paths = []
if TARGET_COL in df.columns:
    p = OUT_DIR / "conteo_target.csv"
    df[TARGET_COL].value_counts(dropna=False).to_csv(p); paths.append(p)
if LOCALIDAD_COL in df.columns:
    p = OUT_DIR / "conteo_localidad.csv"
    df[LOCALIDAD_COL].value_counts(dropna=False).to_csv(p); paths.append(p)

p = OUT_DIR / "resumen_nulos.csv"
null_summary.to_csv(p); paths.append(p)

p = OUT_DIR / "describe_numerico.csv"
df.describe(include=[np.number]).transpose().to_csv(p); paths.append(p)
p = OUT_DIR / "describe_completo.csv"
df.describe(include='all').transpose().to_csv(p); paths.append(p)

p = OUT_DIR / "tipos_de_datos.csv"
df.dtypes.to_frame("dtype").to_csv(p); paths.append(p)

print("Archivos de salida guardados en:")
for p in paths:
    print(" -", p)


In [ ]:
listar_out_dir()  # Se ejecuta para ver lo generado
descargar_zip_outputs()  # Se ejecuta para descargarlo
